# Import libraries

In [ ]:
import pandas as pd
import numpy as np
from matplotlib.backends.backend_tkagg import (
    FigureCanvasTkAgg, NavigationToolbar2Tk)
import io
import random
import matplotlib.pyplot as plt
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor, BaggingRegressor, AdaBoostRegressor
from sklearn.linear_model import LassoCV, LassoLarsCV
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.linear_model import ARDRegression, BayesianRidge
from sklearn.linear_model import GammaRegressor, PoissonRegressor
from sklearn.linear_model import SGDRegressor, RidgeCV
from sklearn.neural_network import MLPRegressor
from sklearn.svm import SVR
from sklearn.model_selection import cross_val_score, cross_validate, cross_val_predict
from sklearn.feature_selection import SelectFromModel
from sklearn.model_selection import KFold
from sklearn.metrics import r2_score
import seaborn as sns
from numpy import mean
from numpy import std
from numpy import sum as arraysum
from numpy import sqrt
import plotly.graph_objects as go
import plotly.express as px
import scipy.stats as st
from matplotlib import pyplot
import pickle
import warnings; warnings.simplefilter('ignore')
from xgboost import XGBRegressor
from jupyterthemes import get_themes
import jupyterthemes as jt
from jupyterthemes.stylefx import set_nb_theme

#from google.colab import files
#from google.colab import drive
#drive.mount('/content/drive')

# Load database

In [2]:
##Output is the data extracted from database
try:
  Output=pd.read_excel (r'/Users/amir/Desktop/PhD/PhD/Data_new/Formatted/output.xlsx')
except:  
  drive.mount('/content/drive')
  Output=pd.read_excel (r'/content/drive/My Drive/PhD/Data_new/Formatted/output.xlsx')
##Material class in the database at this moment are 10 different material
##Curently the material classes are ['BFA', 'Boden', 'EDS', 'GKOS', 'GRS', 'GS', 'HMVA', 'RC', 'SFA', 'SKA']
material_class=eval('Output')['MaterialClass'].unique()
material_class=np.delete(material_class, 0)
material_class=list(material_class)
##Currently we have 6 different test type in our database 
##['DIN19528', 'DIN19529', 'DIN19528EX', 'DIN38414-4', 'CEN/TS16637-3', 'FprEN14405']
test_type=eval('Output')['Test_type'].unique()
test_type=np.delete(test_type, 0)
test_type=list(test_type)


# Data pre-processing

In [3]:
Raw_data=Output
data_back=Output
#Replace None with nan
data_back.iloc[2:, 5:7].replace(to_replace=[None], value=np.nan, inplace=True)
modified_data=data_back.iloc[2:, 7:]
#Replace none with zero
modified_data.replace(to_replace=[None], value=float(0), inplace=True)


In [4]:
#Annoying string eliminator

filter_data = np.column_stack([modified_data[col].str.contains(r"D", na=False) for col in modified_data])
filter_get=modified_data.loc[filter_data.any(axis=1)]
index_filter=(filter_get.index)
modified_data=modified_data.drop([index_filter[0]])

filter_data = np.column_stack([modified_data[col].str.contains(r"<", na=False) for col in modified_data])
find_filter=np.where(filter_data==True)
for i in range (len(find_filter[0])):
    modified_data.iat[find_filter[0][i], find_filter[1][i]]=0

filter_data = np.column_stack([modified_data[col].str.contains(r"-", na=False) for col in modified_data])
find_filter=np.where(filter_data==True)
for i in range (len(find_filter[0])):
    modified_data.iat[find_filter[0][i], find_filter[1][i]]=(random.uniform(0, abs(float(modified_data.iat[find_filter[0][i], find_filter[1][i]]) )))

filter_data = np.column_stack([modified_data[col].str.contains(r"kA", na=False) for col in modified_data])
find_filter=np.where(filter_data==True)
for i in range (len(find_filter[0])):
      modified_data.iat[find_filter[0][i], find_filter[1][i]]=0  

filter_data = np.column_stack([modified_data[col].str.contains(r",", na=False) for col in modified_data])
find_filter=np.where(filter_data==True)
for i in range (len(find_filter[0])):
      modified_data.iat[find_filter[0][i], find_filter[1][i]]=modified_data.iat[find_filter[0][i], find_filter[1][i]].replace(',','.')

filter_data = np.column_stack([modified_data[col].str.contains(r".", na=False) for col in modified_data])
find_filter=np.where(filter_data==True)
for i in range (len(find_filter[0])):
    modified_data.iat[find_filter[0][i], find_filter[1][i]]=modified_data.iat[find_filter[0][i], find_filter[1][i]].replace('..','.') 

filter_data = np.column_stack([modified_data[col].str.contains(r"n.", na=False) for col in modified_data])
find_filter=np.where(filter_data==True)
for i in range (len(find_filter[0])):
    modified_data.iat[find_filter[0][i], find_filter[1][i]]=0  

modified_data = modified_data.astype(float) 
modified_data=modified_data.drop(columns=['PAK','KW_Index','Bromacil_Eluate'])


In [5]:
col=modified_data.columns
for j in range (len(col)):
    lim=float(Raw_data[col[j]].iloc[1])
    if np.isnan(lim)==True:
        find_filter=modified_data[col[j]].loc[modified_data[col[j]]==0].index
        f=modified_data[col[j]][2:]
        ff=f.sort_values( ascending=True).unique()
        lim=ff[1]
        for i in range (len(find_filter)):
            modified_data[col[j]].loc[[find_filter[i]]]=(random.uniform(0, lim)) 
    else:
            find_filter=modified_data[col[j]].loc[modified_data[col[j]]==0].index
            
            for i in range (len(find_filter)):
                modified_data[col[j]].loc[[find_filter[i]]]=(random.uniform(0, lim))   
                
                
    col=modified_data.columns
    for j in range (len(col)):
        lim=float(Raw_data[col[j]].iloc[1])
        find_filter=modified_data[col[j]].loc[modified_data[col[j]]<=0].index
        
        for i in range (len(find_filter)):
            modified_data[col[j]].loc[[find_filter[i]]]=(random.uniform(0, lim))   

In [6]:
machine_data=modified_data
modified_data.insert(0,'ID', Output['ID'])

modified_data.insert(1,'Column_1', Output['Column_1'])

modified_data.insert(2,'Probe', Output['Probe'])

modified_data.insert(3,'ClearName', Output['ClearName'])

modified_data.insert(4,'LS', Output['LS'])

modified_data.insert(5,'Test_type', Output['Test_type'])

modified_data.insert(6,'MaterialClass', Output['MaterialClass'])

modified_data = modified_data.append(Output[Output['ID'] == 0])

modified_data = modified_data.append(Output[Output['ID'] == 1])

modified_data = modified_data.sort_index()
data_back=machine_data.loc[machine_data['Column_1']=='BG' ]
for i in range (len(data_back)):
    machine_data=machine_data.drop([data_back.index[i]])

machine_data=machine_data.drop(columns=['Column_1','ClearName','ID'])


In [7]:
##Convert test type to integer

machine_data=machine_data.replace(to_replace=r'^DIN19528EX', value=3, regex=True)

machine_data=machine_data.replace(to_replace=r'^DIN19528', value=1, regex=True)

machine_data=machine_data.replace(to_replace=r'^DIN19529', value=2, regex=True)

machine_data=machine_data.replace(to_replace=r'^DIN38414-4', value=4, regex=True)

machine_data=machine_data.replace(to_replace=r'^CEN/TS16637-3', value=5, regex=True)

machine_data=machine_data.replace(to_replace=r'^FprEN14405', value=6, regex=True)


In [8]:
##Convert material class to integer

for i in range (len(material_class)):
  machine_data['MaterialClass']=machine_data['MaterialClass'].replace(to_replace=material_class[i], value=i+1, regex=True)

machine_data=machine_data.replace(to_replace=np.nan, value=0, regex=True)

#  New datasets

# Initialize the training and test sets

In [9]:
##Loading the training and test dataset
try:
  X_train=pd.read_excel (r'/Users/amir/Desktop/PhD/Data_new/Formatted/X_train10.xlsx')
  X_test=pd.read_excel (r'/Users/amir/Desktop/PhD/Data_new/Formatted/X_test10.xlsx')
  y_train=pd.read_excel (r'/Users/amir/Desktop/PhD/Data_new/Formatted/y_train10.xlsx')
  y_test=pd.read_excel (r'/Users/amir/Desktop/PhD/Data_new/Formatted/y_test10.xlsx')
  
except:
  X_train=pd.read_excel (r'/Users/amir/Desktop/PhD/PhD/Data_new/Formatted/X_train10.xlsx')
  X_test=pd.read_excel (r'/Users/amir/Desktop/PhD/PhD/Data_new/Formatted/X_test10.xlsx')
  y_train=pd.read_excel (r'/Users/amir/Desktop/PhD/PhD/Data_new/Formatted/y_train10.xlsx')
  y_test=pd.read_excel (r'/Users/amir/Desktop/PhD/PhD/Data_new/Formatted/y_test10.xlsx')
    
#X_train.to_excel("X_train1.xlsx");X_test.to_excel("X_test1.xlsx");y_train.to_excel("y_train1.xlsx");y_test.to_excel("y_test1.xlsx")



# Scenario 1 (all materials and all test types)

In [ ]:
##Remove all column or components which contains sum like sum PAK, since sometimes
##The concentration of related components are not available and we can't use the random value
##Instead of unknown value
try:
    sumcomp = [col for col in X_train.columns if 'Sum' in col]

    for i in range (len(sumcomp)):
        X_train=X_train.drop([sumcomp[i]], axis=1)
        X_test=X_test.drop([sumcomp[i]], axis=1)
        y_test=y_test.drop([sumcomp[i]], axis=1)
    X_train.rename(columns={'Unnamed: 0': 'Index'}, inplace=True)
    X_train=X_train.set_index('Index')
    X_test.rename(columns={'Unnamed: 0': 'Index'}, inplace=True)
    X_test=X_test.set_index('Index')
    y_train.rename(columns={'Unnamed: 0': 'Index'}, inplace=True)
    y_train=y_train.set_index('Index')
    y_test.rename(columns={'Unnamed: 0': 'Index'}, inplace=True)
    y_test=y_test.set_index('Index')
    X_train=X_train.drop(columns=['Probe'])
    X_test=X_test.drop(columns=['Probe'])
    y_test=y_test.drop(columns=['Probe'])
    X_test=(X_test.astype(float))
    X_train=(X_train.astype(float))
    y_test=(y_test.astype(float))

    X_train_back=X_train
    y_train_back=y_train
    y_test_back=y_test
    X_test_back=X_test
except:
    pass
    

####### Get rid of annoying columns and strings

#######Convert the dataset to float


a=X_train_back.columns
a=pd.DataFrame(a)
b=a.loc[a[0]=='pH_y'].index
aa=a[b[0]:]
a=X_train_back.columns
aa=a[b[0]:]

#y_train=X_train[a[first:second]]
#y_test=y_test[a[first:second]]
XX_train=X_train_back.drop(columns=aa)
XX_test=X_test_back.drop(columns=aa)


##Find the related column which our target components are starting
##It means we will seprate the all chemical components results from short term test
##and the LS, material class and test type of the long term test in the training set
##Training set=(all info from short term test+ LS,material class,test type (long term test))
##we need to predict the rest of information for long term test, but one by one

##Define the specific chemical components in order to predict
component = input("Enter your considered chemical component: (Start with capital letter)\n'But --pH-- is written like that'\n ")

try:
    filter_data = np.column_stack(a[XX_train.shape[1]:].str.contains(str(component), na=False) for col in a[XX_train.shape[1]:])
except:
    pass
filter_data=pd.DataFrame(filter_data)
component_index= filter_data.loc[filter_data[0]==True].index[0]

first=component_index+XX_train.shape[1]
second=first+1

X_train=X_train_back
y_train=y_train_back
y_test=y_test_back
X_test=X_test_back    
y_train=X_train[a[first:second]]
y_test=y_test[a[first:second]]
X_train=X_train.drop(columns=aa)
X_test=X_test.drop(columns=aa)


# Scenario 2 (all extended test type)

In [ ]:
##Remove all column or components which contains sum like sum PAK, since sometimes
##The concentration of related components are not available and we can't use the random value
##Instead of unknown value
try:
    sumcomp = [col for col in X_train.columns if 'Sum' in col]

    for i in range (len(sumcomp)):
        X_train=X_train.drop([sumcomp[i]], axis=1)
        X_test=X_test.drop([sumcomp[i]], axis=1)
        y_test=y_test.drop([sumcomp[i]], axis=1)
    X_train.rename(columns={'Unnamed: 0': 'Index'}, inplace=True)
    X_train=X_train.set_index('Index')
    X_test.rename(columns={'Unnamed: 0': 'Index'}, inplace=True)
    X_test=X_test.set_index('Index')
    y_train.rename(columns={'Unnamed: 0': 'Index'}, inplace=True)
    y_train=y_train.set_index('Index')
    y_test.rename(columns={'Unnamed: 0': 'Index'}, inplace=True)
    y_test=y_test.set_index('Index')
    X_train=X_train.drop(columns=['Probe'])
    X_test=X_test.drop(columns=['Probe'])
    y_test=y_test.drop(columns=['Probe'])
    X_test=(X_test.astype(float))
    X_train=(X_train.astype(float))
    y_test=(y_test.astype(float))

    X_train_back=X_train
    y_train_back=y_train
    y_test_back=y_test
    X_test_back=X_test
except:
    pass
    

####### Get rid of annoying columns and strings

#######Convert the dataset to float

a=X_train_back.columns
a=pd.DataFrame(a)
b=a.loc[a[0]=='pH_y'].index
aa=a[b[0]:]
a=X_train_back.columns
aa=a[b[0]:]

XX_train=X_train_back.drop(columns=aa)
XX_test=X_test_back.drop(columns=aa)


##Find the related column which our target components are starting
##It means we will seprate the all chemical components results from short term test
##and the LS, material class and test type of the long term test in the training set
##Training set=(all info from short term test+ LS,material class,test type (long term test))
##we need to predict the rest of information for long term test, but one by one

##Define the specific chemical components in order to predict
component = input("Enter your considered chemical component: (Start with capital letter)\n'But --pH-- is written like that'\n ")

try:
    filter_data = np.column_stack(a[XX_train.shape[1]:].str.contains(str(component), na=False) for col in a[XX_train.shape[1]:])
except:
    pass
filter_data=pd.DataFrame(filter_data)
component_index= filter_data.loc[filter_data[0]==True].index[0]

first=component_index+XX_train.shape[1]
second=first+1

X_train=X_train_back
y_train=y_train_back
y_test=y_test_back
X_test=X_test_back    
y_train=X_train[a[first:second]]
y_test=y_test[a[first:second]]
X_train=X_train.drop(columns=aa)
X_test=X_test.drop(columns=aa)


a=X_train[(X_train['encoder__x0_DIN19528EX_x'] == 1) | (X_train['encoder__x0_FprEN14405_x'] == 1)| (X_train['encoder__x0_CEN/TS16637-3_x'] == 1)].index
X_train=X_train.loc[a]
y_train=y_train.loc[a]

a=X_train[(X_train['encoder__x0_DIN19528EX_y'] == 1) | (X_train['encoder__x0_FprEN14405_y'] == 1)| (X_train['encoder__x0_CEN/TS16637-3_y'] == 1)].index
X_train=X_train.loc[a]
y_train=y_train.loc[a]

b=X_test[(X_test['encoder__x0_DIN19528EX_x'] == 1) | (X_test['encoder__x0_FprEN14405_x'] == 1)| (X_test['encoder__x0_CEN/TS16637-3_x'] == 1)].index
X_test=X_test.loc[b]
y_test=y_test.loc[b]

b=X_test[(X_test['encoder__x0_DIN19528EX_y'] == 1) | (X_test['encoder__x0_FprEN14405_y'] == 1)| (X_test['encoder__x0_CEN/TS16637-3_y'] == 1)].index
X_test=X_test.loc[b]
y_test=y_test.loc[b]

# a=X_train[(X_train['encoder__x0_DIN19528EX_x'] == 1) | (X_train['encoder__x0_FprEN14405_x'] == 1)].index
# X_train=X_train.loc[a]
# y_train=y_train.loc[a]

# a=X_train[(X_train['encoder__x0_DIN19528EX_y'] == 1) | (X_train['encoder__x0_FprEN14405_y'] == 1)].index
# X_train=X_train.loc[a]
# y_train=y_train.loc[a]

# b=X_test[(X_test['encoder__x0_DIN19528EX_x'] == 1) | (X_test['encoder__x0_FprEN14405_x'] == 1)].index
# X_test=X_test.loc[b]
# y_test=y_test.loc[b]

# b=X_test[(X_test['encoder__x0_DIN19528EX_y'] == 1) | (X_test['encoder__x0_FprEN14405_y'] == 1)].index
# X_test=X_test.loc[b]
# y_test=y_test.loc[b]
# aa=np.array(X_trainencoder__x0_BFA__x0_CEN/TS16637-3_x']==1)
# print(aa)
# bb=np.where(aa==True)
# print(np.shape(bb))

# Scenario 3 (all materials contains DIN19528EX)

In [ ]:
##Remove all column or components which contains sum like sum PAK, since sometimes
##The concentration of related components are not available and we can't use the random value
##Instead of unknown value
try:
    sumcomp = [col for col in X_train.columns if 'Sum' in col]

    for i in range (len(sumcomp)):
        X_train=X_train.drop([sumcomp[i]], axis=1)
        X_test=X_test.drop([sumcomp[i]], axis=1)
        y_test=y_test.drop([sumcomp[i]], axis=1)
    X_train.rename(columns={'Unnamed: 0': 'Index'}, inplace=True)
    X_train=X_train.set_index('Index')
    X_test.rename(columns={'Unnamed: 0': 'Index'}, inplace=True)
    X_test=X_test.set_index('Index')
    y_train.rename(columns={'Unnamed: 0': 'Index'}, inplace=True)
    y_train=y_train.set_index('Index')
    y_test.rename(columns={'Unnamed: 0': 'Index'}, inplace=True)
    y_test=y_test.set_index('Index')
    X_train=X_train.drop(columns=['Probe'])
    X_test=X_test.drop(columns=['Probe'])
    y_test=y_test.drop(columns=['Probe'])
    X_test=(X_test.astype(float))
    X_train=(X_train.astype(float))
    y_test=(y_test.astype(float))

    X_train_back=X_train
    y_train_back=y_train
    y_test_back=y_test
    X_test_back=X_test
except:
    pass
    

####### Get rid of annoying columns and strings

#######Convert the dataset to float

a=X_train_back.columns
a=pd.DataFrame(a)
b=a.loc[a[0]=='pH_y'].index
aa=a[b[0]:]
a=X_train_back.columns
aa=a[b[0]:]

XX_train=X_train_back.drop(columns=aa)
XX_test=X_test_back.drop(columns=aa)


##Find the related column which our target components are starting
##It means we will seprate the all chemical components results from short term test
##and the LS, material class and test type of the long term test in the training set
##Training set=(all info from short term test+ LS,material class,test type (long term test))
##we need to predict the rest of information for long term test, but one by one

##Define the specific chemical components in order to predict
component = input("Enter your considered chemical component: (Start with capital letter)\n'But --pH-- is written like that'\n ")

try:
    filter_data = np.column_stack(a[XX_train.shape[1]:].str.contains(str(component), na=False) for col in a[XX_train.shape[1]:])
except:
    pass
filter_data=pd.DataFrame(filter_data)
component_index= filter_data.loc[filter_data[0]==True].index[0]

first=component_index+XX_train.shape[1]
second=first+1

X_train=X_train_back
y_train=y_train_back
y_test=y_test_back
X_test=X_test_back    
y_train=X_train[a[first:second]]
y_test=y_test[a[first:second]]
X_train=X_train.drop(columns=aa)
X_test=X_test.drop(columns=aa)


a=X_train[(X_train['encoder__x0_DIN19528EX_x'] == 1)].index
X_train=X_train.loc[a]
y_train=y_train.loc[a]

a=X_train[(X_train['encoder__x0_DIN19528EX_y'] == 1)].index
X_train=X_train.loc[a]
y_train=y_train.loc[a]

b=X_test[(X_test['encoder__x0_DIN19528EX_x'] == 1)].index
X_test=X_test.loc[b]
y_test=y_test.loc[b]

b=X_test[(X_test['encoder__x0_DIN19528EX_y'] == 1)].index
X_test=X_test.loc[b]
y_test=y_test.loc[b]

# Scenario 4 (all materials contains FprEN14405)

In [ ]:
##Remove all column or components which contains sum like sum PAK, since sometimes
##The concentration of related components are not available and we can't use the random value
##Instead of unknown value
try:
    sumcomp = [col for col in X_train.columns if 'Sum' in col]

    for i in range (len(sumcomp)):
        X_train=X_train.drop([sumcomp[i]], axis=1)
        X_test=X_test.drop([sumcomp[i]], axis=1)
        y_test=y_test.drop([sumcomp[i]], axis=1)
    X_train.rename(columns={'Unnamed: 0': 'Index'}, inplace=True)
    X_train=X_train.set_index('Index')
    X_test.rename(columns={'Unnamed: 0': 'Index'}, inplace=True)
    X_test=X_test.set_index('Index')
    y_train.rename(columns={'Unnamed: 0': 'Index'}, inplace=True)
    y_train=y_train.set_index('Index')
    y_test.rename(columns={'Unnamed: 0': 'Index'}, inplace=True)
    y_test=y_test.set_index('Index')
    X_train=X_train.drop(columns=['Probe'])
    X_test=X_test.drop(columns=['Probe'])
    y_test=y_test.drop(columns=['Probe'])
    X_test=(X_test.astype(float))
    X_train=(X_train.astype(float))
    y_test=(y_test.astype(float))

    X_train_back=X_train
    y_train_back=y_train
    y_test_back=y_test
    X_test_back=X_test
except:
    pass
    

####### Get rid of annoying columns and strings

#######Convert the dataset to float

a=X_train_back.columns
a=pd.DataFrame(a)
b=a.loc[a[0]=='pH_y'].index
aa=a[b[0]:]
a=X_train_back.columns
aa=a[b[0]:]

XX_train=X_train_back.drop(columns=aa)
XX_test=X_test_back.drop(columns=aa)


##Find the related column which our target components are starting
##It means we will seprate the all chemical components results from short term test
##and the LS, material class and test type of the long term test in the training set
##Training set=(all info from short term test+ LS,material class,test type (long term test))
##we need to predict the rest of information for long term test, but one by one

##Define the specific chemical components in order to predict
component = input("Enter your considered chemical component: (Start with capital letter)\n'But --pH-- is written like that'\n ")

try:
    filter_data = np.column_stack(a[XX_train.shape[1]:].str.contains(str(component), na=False) for col in a[XX_train.shape[1]:])
except:
    pass
filter_data=pd.DataFrame(filter_data)
component_index= filter_data.loc[filter_data[0]==True].index[0]

first=component_index+XX_train.shape[1]
second=first+1

X_train=X_train_back
y_train=y_train_back
y_test=y_test_back
X_test=X_test_back    
y_train=X_train[a[first:second]]
y_test=y_test[a[first:second]]
X_train=X_train.drop(columns=aa)
X_test=X_test.drop(columns=aa)


a=X_train[(X_train['encoder__x0_FprEN14405_x'] == 1)].index
X_train=X_train.loc[a]
y_train=y_train.loc[a]

a=X_train[(X_train['encoder__x0_FprEN14405_y'] == 1)].index
X_train=X_train.loc[a]
y_train=y_train.loc[a]

b=X_test[(X_test['encoder__x0_FprEN14405_x'] == 1)].index
X_test=X_test.loc[b]
y_test=y_test.loc[b]

b=X_test[(X_test['encoder__x0_FprEN14405_y'] == 1)].index
X_test=X_test.loc[b]
y_test=y_test.loc[b]

# Scenario 5 (all materials contains CEN/TS16637-3)

In [ ]:
##Remove all column or components which contains sum like sum PAK, since sometimes
##The concentration of related components are not available and we can't use the random value
##Instead of unknown value
try:
    sumcomp = [col for col in X_train.columns if 'Sum' in col]

    for i in range (len(sumcomp)):
        X_train=X_train.drop([sumcomp[i]], axis=1)
        X_test=X_test.drop([sumcomp[i]], axis=1)
        y_test=y_test.drop([sumcomp[i]], axis=1)
    X_train.rename(columns={'Unnamed: 0': 'Index'}, inplace=True)
    X_train=X_train.set_index('Index')
    X_test.rename(columns={'Unnamed: 0': 'Index'}, inplace=True)
    X_test=X_test.set_index('Index')
    y_train.rename(columns={'Unnamed: 0': 'Index'}, inplace=True)
    y_train=y_train.set_index('Index')
    y_test.rename(columns={'Unnamed: 0': 'Index'}, inplace=True)
    y_test=y_test.set_index('Index')
    X_train=X_train.drop(columns=['Probe'])
    X_test=X_test.drop(columns=['Probe'])
    y_test=y_test.drop(columns=['Probe'])
    X_test=(X_test.astype(float))
    X_train=(X_train.astype(float))
    y_test=(y_test.astype(float))

    X_train_back=X_train
    y_train_back=y_train
    y_test_back=y_test
    X_test_back=X_test
except:
    pass
    

####### Get rid of annoying columns and strings

#######Convert the dataset to float

a=X_train_back.columns
a=pd.DataFrame(a)
b=a.loc[a[0]=='pH_y'].index
aa=a[b[0]:]
a=X_train_back.columns
aa=a[b[0]:]

XX_train=X_train_back.drop(columns=aa)
XX_test=X_test_back.drop(columns=aa)


##Find the related column which our target components are starting
##It means we will seprate the all chemical components results from short term test
##and the LS, material class and test type of the long term test in the training set
##Training set=(all info from short term test+ LS,material class,test type (long term test))
##we need to predict the rest of information for long term test, but one by one

##Define the specific chemical components in order to predict
component = input("Enter your considered chemical component: (Start with capital letter)\n'But --pH-- is written like that'\n ")

try:
    filter_data = np.column_stack(a[XX_train.shape[1]:].str.contains(str(component), na=False) for col in a[XX_train.shape[1]:])
except:
    pass
filter_data=pd.DataFrame(filter_data)
component_index= filter_data.loc[filter_data[0]==True].index[0]

first=component_index+XX_train.shape[1]
second=first+1

X_train=X_train_back
y_train=y_train_back
y_test=y_test_back
X_test=X_test_back    
y_train=X_train[a[first:second]]
y_test=y_test[a[first:second]]
X_train=X_train.drop(columns=aa)
X_test=X_test.drop(columns=aa)


a=X_train[(X_train['encoder__x0_CEN/TS16637-3_x'] == 1)].index
X_train=X_train.loc[a]
y_train=y_train.loc[a]

a=X_train[(X_train['encoder__x0_CEN/TS16637-3_y'] == 1)].index
X_train=X_train.loc[a]
y_train=y_train.loc[a]

b=X_test[(X_test['encoder__x0_CEN/TS16637-3_x'] == 1)].index
X_test=X_test.loc[b]
y_test=y_test.loc[b]

b=X_test[(X_test['encoder__x0_CEN/TS16637-3_y'] == 1)].index
X_test=X_test.loc[b]
y_test=y_test.loc[b]

# Scenario 6 (RC and all test types)

In [122]:
##Remove all column or components which contains sum like sum PAK, since sometimes
##The concentration of related components are not available and we can't use the random value
##Instead of unknown value
try:
    sumcomp = [col for col in X_train.columns if 'Sum' in col]

    for i in range (len(sumcomp)):
        X_train=X_train.drop([sumcomp[i]], axis=1)
        X_test=X_test.drop([sumcomp[i]], axis=1)
        y_test=y_test.drop([sumcomp[i]], axis=1)
    X_train.rename(columns={'Unnamed: 0': 'Index'}, inplace=True)
    X_train=X_train.set_index('Index')
    X_test.rename(columns={'Unnamed: 0': 'Index'}, inplace=True)
    X_test=X_test.set_index('Index')
    y_train.rename(columns={'Unnamed: 0': 'Index'}, inplace=True)
    y_train=y_train.set_index('Index')
    y_test.rename(columns={'Unnamed: 0': 'Index'}, inplace=True)
    y_test=y_test.set_index('Index')
    X_train=X_train.drop(columns=['Probe'])
    X_test=X_test.drop(columns=['Probe'])
    y_test=y_test.drop(columns=['Probe'])
    X_test=(X_test.astype(float))
    X_train=(X_train.astype(float))
    y_test=(y_test.astype(float))

    X_train_back=X_train
    y_train_back=y_train
    y_test_back=y_test
    X_test_back=X_test
except:
    pass
    

####### Get rid of annoying columns and strings

#######Convert the dataset to float

a=X_train_back.columns
a=pd.DataFrame(a)
b=a.loc[a[0]=='pH_y'].index

aa=a[b[0]:]
a=X_train_back.columns
aa=a[b[0]:]

XX_train=X_train_back.drop(columns=aa)
XX_test=X_test_back.drop(columns=aa)


##Find the related column which our target components are starting
##It means we will seprate the all chemical components results from short term test
##and the LS, material class and test type of the long term test in the training set
##Training set=(all info from short term test+ LS,material class,test type (long term test))
##we need to predict the rest of information for long term test, but one by one

##Define the specific chemical components in order to predict
component = input("Enter your considered chemical component: (Start with capital letter)\n'But --pH-- is written like that'\n ")

try:
    filter_data = np.column_stack(a[XX_train.shape[1]:].str.contains(str(component), na=False) for col in a[XX_train.shape[1]:])
except:
    pass
filter_data=pd.DataFrame(filter_data)
component_index= filter_data.loc[filter_data[0]==True].index[0]

first=component_index+XX_train.shape[1]
second=first+1

X_train=X_train_back
y_train=y_train_back
y_test=y_test_back
X_test=X_test_back    
y_train=X_train[a[first:second]]
y_test=y_test[a[first:second]]
X_train=X_train.drop(columns=aa)
X_test=X_test.drop(columns=aa)


a=X_train[(X_train['encoder__x0_DIN19528EX_x'] == 1) | (X_train['encoder__x0_FprEN14405_x'] == 1)| (X_train['encoder__x0_CEN/TS16637-3_x'] == 1)].index
X_train=X_train.loc[a]
y_train=y_train.loc[a]

a=X_train[(X_train['encoder__x0_DIN19528EX_y'] == 1) | (X_train['encoder__x0_FprEN14405_y'] == 1)| (X_train['encoder__x0_CEN/TS16637-3_y'] == 1)].index
X_train=X_train.loc[a]
y_train=y_train.loc[a]

b=X_test[(X_test['encoder__x0_DIN19528EX_x'] == 1) | (X_test['encoder__x0_FprEN14405_x'] == 1)| (X_test['encoder__x0_CEN/TS16637-3_x'] == 1)].index
X_test=X_test.loc[b]
y_test=y_test.loc[b]

b=X_test[(X_test['encoder__x0_DIN19528EX_y'] == 1) | (X_test['encoder__x0_FprEN14405_y'] == 1)| (X_test['encoder__x0_CEN/TS16637-3_y'] == 1)].index
X_test=X_test.loc[b]
y_test=y_test.loc[b]


# a=X_train[(X_train['encoder__x0_DIN19528EX_x'] == 1)].index
# X_train=X_train.loc[a]
# y_train=y_train.loc[a]

# a=X_train[(X_train['encoder__x0_DIN19528EX_y'] == 1)].index
# X_train=X_train.loc[a]
# y_train=y_train.loc[a]

# b=X_test[(X_test['encoder__x0_DIN19528EX_x'] == 1)].index
# X_test=X_test.loc[b]
# y_test=y_test.loc[b]

# b=X_test[(X_test['encoder__x0_DIN19528EX_y'] == 1)].index
# X_test=X_test.loc[b]
# y_test=y_test.loc[b]

a=X_train[(X_train['encoder__x0_RC'] == 1)].index
#a=X_train[(X_train['encoder__x0_RC'] == 1) | (X_train['encoder__x0_HMVA'] == 1)].index
X_train=X_train.loc[a]
y_train=y_train.loc[a]

b=X_test[(X_test['encoder__x0_RC'] == 1)].index
# b=X_test[(X_test['encoder__x0_RC'] == 1)| (X_test['encoder__x0_HMVA'] == 1)].index
X_test=X_test.loc[b]
y_test=y_test.loc[b]

# from sklearn.preprocessing import StandardScaler
# sc_x=StandardScaler()
# sc_xx=StandardScaler()
# sc_y=StandardScaler()
# sc_yy=StandardScaler()
# X_train=sc_x.fit_transform(X_train)
# X_test=sc_x.fit_transform(X_test)
# y_train=sc_y.fit_transform(y_train)
# y_test=sc_yy.fit_transform(y_test)


Enter your considered chemical component: (Start with capital letter)
'But --pH-- is written like that'
 Sili


# Implement machine learning algorithm and evaluating the model performance

In [123]:
##Implement the machine learning algorithm

##Select algorithms

##Accuracy of the model
def evaluate():
  r2_traine=mode.score(X_train, y_train)
  print('Train score')
  print(r2_traine)
  r2_teste=mode.score(X_test, y_test)
  print('Test score')
  print(r2_teste)
  globals()['predictionse_'+regressors[i]]=mode.predict(X_test)


regressors = ['RandomForest','ExtraTrees','ARD','BayesianRidge','LassoLarsCV','GradientBoosting','BaggingRegressor','XGboost']
#regressors = ['RandomForest','ExtraTrees','XGboost']

for i in range (len(regressors)):
  if regressors[i] == 'RandomForest':
    mode = RandomForestRegressor(n_estimators=400, random_state=1, n_jobs=-1).fit(X_train, y_train)
    print('\n'+regressors[i])
    evaluate()
    filename = y_test.columns[0]+regressors[i]+'.sav'
    pickle.dump(mode, open(filename, 'wb'))
  if regressors[i] == 'ExtraTrees':
    mode = ExtraTreesRegressor(n_estimators=400, random_state=1, n_jobs=-1).fit(X_train, y_train)
    print('\n'+regressors[i])
    evaluate()
    filename = y_test.columns[0]+regressors[i]+'.sav'
    pickle.dump(mode, open(filename, 'wb'))
  if regressors[i] == 'ARD':
    mode = ARDRegression(n_iter=500).fit(X_train, y_train)
    print('\n'+regressors[i])
    evaluate()
    filename = y_test.columns[0]+regressors[i]+'.sav'
    pickle.dump(mode, open(filename, 'wb'))
  if regressors[i] == 'BayesianRidge':
    mode = BayesianRidge(n_iter=600).fit(X_train, y_train)
    print('\n'+regressors[i])
    evaluate()  
    filename = y_test.columns[0]+regressors[i]+'.sav'
    pickle.dump(mode, open(filename, 'wb'))
  if regressors[i] == 'LassoLarsCV':
    mode =  LassoLarsCV(cv=10, normalize=False).fit(X_train, y_train)
    print('\n'+regressors[i])
    evaluate()    
    filename = y_test.columns[0]+regressors[i]+'.sav'
    pickle.dump(mode, open(filename, 'wb'))
  if regressors[i] == 'XGboost':
    mode =  XGBRegressor(n_estimators=600, n_jobs=-1, random_state=1).fit(X_train, y_train)
    print('\n'+regressors[i])
    evaluate()    
    filename = y_test.columns[0]+regressors[i]+'.sav'
    pickle.dump(mode, open(filename, 'wb'))  
  if regressors[i] == 'BaggingRegressor':
    mode =  BaggingRegressor(n_estimators=400, random_state=1, n_jobs=-1).fit(X_train, y_train)
    print('\n'+regressors[i])
    evaluate()    
    filename = y_test.columns[0]+regressors[i]+'.sav'
    pickle.dump(mode, open(filename, 'wb'))    
  if regressors[i] == 'GradientBoosting':
    mode =  GradientBoostingRegressor(n_estimators=600, validation_fraction=0.1,
                                                           random_state=1, criterion="friedman_mse").fit(X_train, y_train)
    print('\n'+regressors[i])
    evaluate()    
    filename = y_test.columns[0]+regressors[i]+'.sav'
    pickle.dump(mode, open(filename, 'wb'))   
    
    



RandomForest
Train score
0.9689685441499664
Test score
0.4548517059812174

ExtraTrees
Train score
0.9999999987711576
Test score
0.5255437499054982

ARD
Train score
0.6183232875123577
Test score
0.5030458542248573

BayesianRidge
Train score
0.5001928409411778
Test score
0.5445360404421824

LassoLarsCV
Train score
0.49390614253207155
Test score
0.5481499917730395

GradientBoosting
Train score
0.9921391497863512
Test score
0.5835008123184919

BaggingRegressor
Train score
0.9689127836802603
Test score
0.45818481114988063

XGboost
Train score
0.999999999999833
Test score
0.5158129509923138


# Cross validation

In [124]:
##Cross validation
X=(pd.concat([X_train,X_test]))
Y=(pd.concat([y_train,y_test]))
kf = KFold(n_splits=10, random_state=1, shuffle=True);

for i in range (len(regressors)):
    mode=pickle.load(open(y_test.columns[0]+regressors[i]+'.sav', 'rb'));
    globals()['r2_traine_'+regressors[i]]=round(mode.score(X_train, y_train),2);
    globals()['test_r_'+regressors[i]]=round(mode.score(X_test, y_test),2);
    globals()['cv_results_'+regressors[i]] = cross_validate(mode, X, Y, cv=kf);
    globals()['train_r_'+regressors[i]]=round(np.mean(eval('cv_results_'+regressors[i])['test_score'],),2);
    globals()['annotations_'+regressors[i]] = [dict(text="$Train Cross Validation R^2="+str(eval('train_r_'+regressors[i]))+'$',
                      xref="paper", yref="paper",
                      x=0.02, y=0.89, showarrow=False),
                        dict(text="$Test R^2="+str(eval('test_r_'+regressors[i]))+'$',
                      xref="paper", yref="paper",
                      x=0.02, y=0.97, showarrow=False),
                        dict(text="$Train R^2="+str(eval('r2_traine_'+regressors[i]))+'$',
                      xref="paper", yref="paper",
                      x=0.02, y=0.81, showarrow=False)]; 
    print('CV '+regressors[i]+'= '+str(eval('train_r_'+regressors[i])))


CV RandomForest= 0.74
CV ExtraTrees= 0.66
CV ARD= 0.6
CV BayesianRidge= 0.5
CV LassoLarsCV= 0.5
CV GradientBoosting= 0.85
CV BaggingRegressor= 0.74
CV XGboost= 0.85


# Visualization

In [ ]:

# predictionse=mode.predict(X_test) 
# ax = sns.regplot(y_test, predictionse, ci=95)
# ax.set_xlabel("Measured")
# ax.set_ylabel("Predicted")
# ax.set_title(y_test.columns[0])

# sum_errs = arraysum((y_test.values - predictionse)**2)
# stdev = sqrt(1/(len(y_test)-2) * sum_errs)
# stdev = np.std(predictionse)

In [117]:
import plotly.graph_objects as pxx
try:
    compname=y_test.columns[0].replace('_y', '')
except:
    pass
for i in range (len(regressors)):
    if max(y_test.values)[0]>max(eval('predictionse_'+regressors[i])):
      yy=y_test
    else:  
      yy=eval('predictionse_'+regressors[i])
    fig = go.Figure()
    try:
        maxi=max(yy.values)
    except:
        maxi=max(yy)
    fig=pxx.Figure(data=[go.Scatter(
        name=regressors[0],
        x=y_test[y_test.columns[0]].values,
        y=(pickle.load(open(y_test.columns[0]+regressors[0]+'.sav', 'rb')).predict(X_test)) , mode='markers'
    ),
        go.Scatter(
        name=regressors[1],
        x=y_test[y_test.columns[0]].values,
        y=(pickle.load(open(y_test.columns[0]+regressors[1]+'.sav', 'rb')).predict(X_test)) , mode='markers'
    ), go.Scatter(
        name=regressors[7],
        x=y_test[y_test.columns[0]].values,
        y=(pickle.load(open(y_test.columns[0]+regressors[7]+'.sav', 'rb')).predict(X_test)) , mode='markers'
    ), go.Scatter(
        name=regressors[5],
        x=y_test[y_test.columns[0]].values,
        y=abs(pickle.load(open(y_test.columns[0]+regressors[5]+'.sav', 'rb')).predict(X_test)) , mode='markers'
    ),
        go.Scatter(x=[0, float(maxi)], y=[0, float(maxi)], name='Line',
                          line = dict(color='black', width=4, dash='dash'))
    ])


annotations_all = [dict(text=" ",
                  xref="paper", yref="paper",
                  x=0.02, y=0.89, showarrow=False)]   
# Add dropdown
fig.update_layout(
    updatemenus=[
        dict(
            
            buttons=list([
                dict(label=regressors[0],
                     method="update",
                     args=[{"visible": [True, False, False, False,True]},
                           {"title": regressors[0]+'<br>'+compname,
                            "annotations": annotations_RandomForest
                            }]),
                dict(label=regressors[1],
                     method="update",
                     args=[{"visible": [False, True, False, False,True]},
                           {"title": regressors[1]+'<br>'+compname,
                            "annotations": annotations_ExtraTrees
                            }]),dict(label=regressors[5],
                     method="update",
                     args=[{"visible": [False, False, False, True,True]},
                           {"title": regressors[5]+'<br>'+compname,
                            "annotations": annotations_GradientBoosting
                            }]),dict(label=regressors[7],
                     method="update",
                     args=[{"visible": [False, False, True, False,True]},
                           {"title": regressors[7]+'<br>'+compname,
                            "annotations": annotations_XGboost
                            }]),dict(label='All',
                     method="update",
                     args=[{"visible": [True,True, True,True,True,True]},
                           {"title": 'All algorithms'+'<br>'+compname,
                            "annotations": annotations_all
                            }])
            ]),
            direction="down",
            pad={"r": 10, "t": 10},
            showactive=True,
            x=0,
            xanchor="left",
            y=1.13,
            yanchor="top",
        )
    ])

fig.update_layout(
    title={
        'y':0.9,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'})

fig.update_xaxes(title_text='Measured')
fig.update_yaxes(title_text='Prediction')

fig.show()


# Visualization by error bar


In [ ]:
# # Create figure
# import plotly.express as px
# r2_traine=mode.score(X_train, y_train)
# r2_teste=mode.score(X_test, y_test)
# print(r2_teste)
# predictionse=mode.predict(X_test) 

# predictionse=pd.DataFrame(predictionse)
# frames = [y_test, predictionse]
# result = pd.concat(frames, axis=1)
# if max(y_test.values)[0]>max(predictionse):
#   yy=y_test
# else:  
#   yy=predictionse
# fig = go.Figure()
# maxi=max(yy.values)
# result.rename(columns={result.columns[0]: 'Measured', 0: 'Prediction'}, inplace=True)


# #fig.add_trace(go.Scatter(x=[0, float(maxi)], y=[0, float(maxi)], name='',
#                         # line = dict(color='black', width=4, dash='dash')))
# # Add traces, one for each slider step
# fig=px.scatter(result, x='Measured', y='Prediction',range_x=[-(float(maxi)/10), float(maxi)+(float(maxi)/10)], range_y=[-(float(maxi)/10),float(maxi)+(float(maxi)/10) ])
# x_in = y_test.values
# y_out = y_test.values
# yhat_out = predictionse.values

# interval = 1 * stdev
# for step in np.arange(0, len(X_test), 1):
#     fig.add_trace(
#         go.Scatter(
#         x=x_in[step],
#         y=yhat_out[step],
#         name='Error bar',
#         error_y=dict(
#             type='data', # value of error bar given in data coordinates
#             array=np.array(interval),
#             visible=True)
   
#     ))

# # Create and add slider
# steps = []
# for i in range(len(X_test)):
#     step = dict(
#         method="update",
#         args=[{"visible": [False] * len(X_test)}
#               ],  # layout attribute
#     )
#     step["args"][0]["visible"][i] = True  # Toggle i'th trace to "visible"
#     steps.append(step)
#     fig.add_trace(go.Scatter(x=result['Measured'], y=result['Prediction'], name='Data points', mode='markers'))
#     fig.add_trace(go.Scatter(x=[0, float(maxi)], y=[0, float(maxi)], name='Line',
#                       line = dict(color='black', width=4, dash='dash')))
#     fig.update_layout(
#     title={
#         'text': y_test.columns[0],
#         'y':0.95,
#         'x':0.5,
#         'xanchor': 'center',
#         'yanchor': 'top'})

# sliders = [dict(
#     active=10,
#     currentvalue={"prefix": "Frequency: "},
#     pad={"t": 50},
#     steps=steps
# )]

# fig.update_layout(
#     sliders=sliders
# )


# fig.show()

